# Smart Interactive Fractal Visualizer (Mandelbrot / Julia)
Shows Re(c) and Im(c) sliders only when Julia set is selected.

In [6]:
import matplotlib.pyplot as plt
from ipywidgets import VBox, HBox, Button, Dropdown, IntSlider, FloatSlider, Output
from IPython.display import display, clear_output
from fraktaly.mandelbrot import compute_mandelbrot_set
from fraktaly.julia import compute_julia_set

out = Output()

fractal_type = Dropdown(options=['Mandelbrot', 'Julia'], value='Mandelbrot', description='Fractal')
iter_slider = IntSlider(value=100, min=10, max=500, step=10, description='Iterace')
re_slider = FloatSlider(value=-0.4, min=-1.0, max=1.0, step=0.01, description='Re(c)')
im_slider = FloatSlider(value=0.6, min=-1.0, max=1.0, step=0.01, description='Im(c)')
zoom_in_btn = Button(description="Zoom In")
zoom_out_btn = Button(description="Zoom Out")
reset_btn = Button(description="Reset")

view = {'xmin': -2.0, 'xmax': 1.5, 'ymin': -1.5, 'ymax': 1.5}
zoom_factor = 0.8

def draw():
    with out:
        clear_output(wait=True)
        width = 600
        height = 600
        if fractal_type.value == 'Mandelbrot':
            data = compute_mandelbrot_set(view['xmin'], view['xmax'], view['ymin'], view['ymax'], width, height, iter_slider.value)
            title = "Mandelbrot Set"
        else:
            c = complex(re_slider.value, im_slider.value)
            data = compute_julia_set(c, view['xmin'], view['xmax'], view['ymin'], view['ymax'], width, height, iter_slider.value)
            title = f"Julia Set (c = {c})"

        plt.figure(figsize=(6,6))
        plt.imshow(data, extent=(view['xmin'], view['xmax'], view['ymin'], view['ymax']), cmap="hot", origin="lower")
        plt.title(title)
        plt.xlabel("Re")
        plt.ylabel("Im")
        plt.show()

def zoom_in(b=None):
    x_center = (view['xmin'] + view['xmax']) / 2
    y_center = (view['ymin'] + view['ymax']) / 2
    x_range = (view['xmax'] - view['xmin']) * zoom_factor / 2
    y_range = (view['ymax'] - view['ymin']) * zoom_factor / 2
    view['xmin'] = x_center - x_range
    view['xmax'] = x_center + x_range
    view['ymin'] = y_center - y_range
    view['ymax'] = y_center + y_range
    draw()

def zoom_out(b=None):
    x_center = (view['xmin'] + view['xmax']) / 2
    y_center = (view['ymin'] + view['ymax']) / 2
    x_range = (view['xmax'] - view['xmin']) / zoom_factor / 2
    y_range = (view['ymax'] - view['ymin']) / zoom_factor / 2
    view['xmin'] = x_center - x_range
    view['xmax'] = x_center + x_range
    view['ymin'] = y_center - y_range
    view['ymax'] = y_center + y_range
    draw()

def reset(b=None):
    view['xmin'], view['xmax'] = -2.0, -1.0
    view['ymin'], view['ymax'] = -1.5, 1.5
    draw()

zoom_in_btn.on_click(zoom_in)
zoom_out_btn.on_click(zoom_out)
reset_btn.on_click(reset)
fractal_type.observe(lambda change: update_ui(), names="value")
iter_slider.observe(lambda change: draw(), names="value")
re_slider.observe(lambda change: draw(), names="value")
im_slider.observe(lambda change: draw(), names="value")

def update_ui():
    if fractal_type.value == 'Julia':
        controls_box.children = [fractal_type, iter_slider, re_slider, im_slider]
    else:
        controls_box.children = [fractal_type, iter_slider]
    draw()

controls_box = VBox()
buttons = HBox([zoom_in_btn, zoom_out_btn, reset_btn])
ui = VBox([controls_box, buttons, out])

display(ui)
update_ui()
